<a href="https://colab.research.google.com/github/kirilexe/Neural_Network_MNIST/blob/main/MNIST_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Network Creating 
Things used here: GPU, Datasets, DataLoader, Transforms, Neural Net, Training & Evaulation

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# hyper params
input_size = 784 # 28x28 (images)
hidden_size = 500
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.0001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)
test_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=False,
                                           transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True
                                           )
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=False
                                           )

examples = iter(test_loader)
example_data, example_targets = next(examples)

for i in range(6):
  plt.subplot(2,3,i+1)
  plt.imshow(example_data[i][0], cmap='gray')
plt.show()

In [ ]:
import torch
import torch.nn as nn

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step[{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

In [ ]:
# Test the model: we don't need to compute gradients
with torch.no_grad():
  n_correct = 0
  n_samples = len(test_loader.dataset)

  for images, labels in test_loader:
    images = images.reshape(-1, 28*28).to(device)
    labels = labels.to(device)

    outputs = model(images)

    # max returns
    _, predicted = torch.max(outputs, 1)
    n_correct += (predicted == labels).sum().item()

acc = n_correct / n_samples
print(f'Accuracy of the network on the {n_samples} test images: {100*acc}%')

In [ ]:
# save the model to a basic pth file
model_filename = 'mnist_model.pth'

print(f"Training complete. Saving model to {model_filename}")
torch.save(model.state_dict(), model_filename)

print("Model state_dict saved successfully!")

In [ ]:
pip install onnx

In [ ]:
# save to a graph
import torch
from torchviz import make_dot #imports that library

dummy_input = torch.randn(1, 1, 28, 28).to(device)

# a forward pass to generate the graph
output = model(dummy_input)
dot = make_dot(output, params=dict(model.named_parameters()))

# save the graph
dot.render("model_graph", format="png", cleanup=True)

print("graph exported to model_graph.png")

In [ ]:
# todo gradio app that runs the model when you draw an image